# Adding Usage Requirements for a Library

Usage requirements of a target parameters allow for far better control over a library or executable's link and include line while also giving more control over the transitive property of targets inside CMake.

In this exercise, we will refactor our code from Adding a Library to use the modern CMake approach. 

We will let our library define its own usage requirements so they are passed transitively to other targets as necessary. 

In this case, MathFunctions will specify any needed include directories itself. Then, the consuming target Tutorial simply needs to link to MathFunctions and not worry about any additional include directories.

First, add a call to target_include_directories() in MathFunctions/CMakeLists. Remember that CMAKE_CURRENT_SOURCE_DIR is the path to the source directory currently being processed.

Then, update (and simplify!) the call to target_include_directories() in the top-level CMakeLists.txt.

## Steps

We want to state that anybody linking to MathFunctions needs to include the current source directory, while MathFunctions itself doesn't. This can be expressed with an INTERFACE usage requirement. Remember INTERFACE means things that consumers require but the producer doesn't.

1. At the end of MathFunctions/CMakeLists.txt, use target_include_directories() with the INTERFACE keyword, as follows:

In [ ]:
# TODO 1: State that anybody linking to MathFunctions needs to include the
# current source directory, while MathFunctions itself doesn't.
# Hint: Use target_include_directories with the INTERFACE keyword
target_include_directories(MathFunctions
                           INTERFACE ${CMAKE_CURRENT_SOURCE_DIR}
                           )

2. Now that we've specified usage requirements for MathFunctions we can safely remove our uses of the EXTRA_INCLUDES variable from the top-level CMakeLists.txt.

Remove this line:

In [ ]:
# TODO 2: Remove EXTRA_INCLUDES list

# add the MathFunctions library
add_subdirectory(MathFunctions)
#list(APPEND EXTRA_INCLUDES "${PROJECT_SOURCE_DIR}/MathFunctions")

3. And remove EXTRA_INCLUDES from target_include_directories:

In [ ]:
# TODO 3: Remove use of EXTRA_INCLUDES

# add the binary tree to the search path for include files
# so that we will find TutorialConfig.h
target_include_directories(Tutorial PUBLIC
                           "${PROJECT_BINARY_DIR}"
                           )

Notice that with this technique, the only thing our executable target does to use our library is call target_link_libraries() with the name of the library target. In larger projects, the classic method of specifying library dependencies manually becomes very complicated very quickly.

# Setting the C++ Standard with Interface Libraries

Now that we have switched our code to a more modern approach, let's demonstrate a modern technique to set properties to multiple targets.

Let's refactor our existing code to use an INTERFACE library. We will use that library in the next step to demonstrate a common use for generator expressions.

In this exercise, we will refactor our code to use an INTERFACE library to specify the C++ standard.

Start this exercise from what we left at the end of Step3 exercise 1. You will have to complete TODO 4 through TODO 7.

Start by editing the top level CMakeLists.txt file. Construct an INTERFACE library target called tutorial_compiler_flags and specify cxx_std_11 as a target compiler feature.

Modify CMakeLists.txt and MathFunctions/CMakeLists.txt so that all targets have a target_link_libraries() call to tutorial_compiler_flags.

Let's update our code from the previous step to use interface libraries to set our C++ requirements.

To start, we need to remove the two set() calls on the variables CMAKE_CXX_STANDARD and CMAKE_CXX_STANDARD_REQUIRED. The specific lines to remove are as follows:

In [ ]:
set(CMAKE_CXX_STANDARD 11)
set(CMAKE_CXX_STANDARD_REQUIRED True)

4. Next, we need to create an interface library, tutorial_compiler_flags. And then use target_compile_features() to add the compiler feature cxx_std_11.

In [ ]:
# TODO 4: Replace the following code by:
# * Creating an interface library called tutorial_compiler_flags
#   Hint: use add_library() with the INTERFACE signature
# * Add compiler feature cxx_std_11 to tutorial_compiler_flags
#   Hint: Use target_compile_features()
add_library(tutorial_compiler_flags INTERFACE)
target_compile_features(tutorial_compiler_flags INTERFACE cxx_std_11)

5. Finally, with our interface library set up, we need to link our executable Tutorial, our SqrtLibrary library and our MathFunctions library to our new tutorial_compiler_flags library. Respectively, the code will look like this:

In [ ]:
# TODO 5: Link Tutorial to tutorial_compiler_flags
target_link_libraries(Tutorial PUBLIC MathFunctions tutorial_compiler_flags)
target_link_libraries(SqrtLibrary PUBLIC tutorial_compiler_flags)
target_link_libraries(MathFunctions PUBLIC tutorial_compiler_flags)

With this, all of our code still requires C++ 11 to build. Notice though that with this method, it gives us the ability to be specific about which targets get specific requirements. In addition, we create a single source of truth in our interface library.